This notebook is for converting a Keras Model to a saved model using a `Saver`

Saving a model usually means saving its variable values to disk.

If you use `Saver`, it will *only* save the variable values in a checkpoint.

If you use `SavedModel`, it also saves the graph and "signatures" which tell consumers of the `SavedModel` (such as tensorflow/serving) how it can be used (which inputs/outputs to apply), and any other files you need to use the model (assets, e.g., vocabulary files).

A model saved in either of these ways can still be trained (with `Saver`, you have to have the original python code. It is possible to train a `SavedModel` directly, although most users I have seen still use the original python code to continue training).

Freezing a model means to replace variables in the graph with constants.

## Setup

In [1]:
%%bash

echo 'Train:'
TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f | sort
echo
echo 'Convert (Saver):'
CONVERT_TF_SAVER_DIR=../../../tmp/trt_end_to_end/convert/tf/saver
find ${CONVERT_TF_SAVER_DIR} -type f | sort
echo
echo 'Convert (Saved Graph):'
CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved_graph
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

Train:
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.h5
../../../tmp/trt_end_to_end/train/basic/001/basic_epoch001_2019-09-03T19:15.md
../../../tmp/trt_end_to_end/train/basic/001/params.json
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.h5
../../../tmp/trt_end_to_end/train/batchn/001/batchn_epoch001_2019-09-03T19:28.md
../../../tmp/trt_end_to_end/train/batchn/001/params.json
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.h5
../../../tmp/trt_end_to_end/train/conv/001/conv_epoch001_2019-09-03T19:30.md
../../../tmp/trt_end_to_end/train/conv/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/params.json
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5
../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.md

Convert (Saver):
../../../tmp/trt_end_to_end/convert/tf/saver/resnet50/001/checkpoint
../../../tmp/trt_end_to_end/convert

In [2]:
import sys
sys.path.append('../../..')

## Parameters

In [3]:
_NAME = 'resnet50'
_EPOCH = 1

In [4]:
from src.utils.trt_end_to_end_constants import *
_NAME, _EPOCH, _TIME = get_params(_NAME, _EPOCH)

In [5]:
_train_filename = H5_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
_train_save_to_dir = get_train_dir(_NAME, _EPOCH)
_train_filepath = os.path.join(_train_save_to_dir, _train_filename)
assert os.path.exists(_train_filepath), "Not exists: %s" % _train_filepath

## Set learning phase 0

In [6]:
# import the needed libraries
import tensorflow as tf
print(tf.__name__, tf.__version__, sep='-')

tf.keras.backend.set_learning_phase(0) # use this if we have batch norm layer in our network

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

tensorflow-1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

## Load keras model

In [7]:
model_filename = H5_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
model_filepath = os.path.join(_train_save_to_dir, model_filename)
print(model_filepath)

keras_model = tf.keras.models.load_model(model_filepath)

W0903 23:51:20.814502 140542846830400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 23:51:20.815482 140542846830400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 23:51:20.818374 140542846830400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

../../../tmp/trt_end_to_end/train/resnet50/001/resnet50_epoch001_2019-09-03T19:31.h5


W0903 23:51:22.833275 140542846830400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Save tensorflow model using `SavedGraphBuilder`

In [10]:
saved_graph_dir = get_saved_graph_dir(_NAME, _EPOCH)

In [11]:
%%bash

CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved_graph
mkdir -p ${CONVERT_TF_SAVED_GRAPH_DIR}
#rm -f ./${CONVERT_TF_SAVER_DIR}/*
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/saved_model.pb
../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/variables/variables.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/variables/variables.index


In [12]:
shutil.rmtree(saved_graph_dir)
os.makedirs(saved_graph_dir)

builder = tf.saved_model.builder.SavedModelBuilder(saved_graph_dir)

# signature = predict_signature_def(inputs = {'input1': model.input[0],
#                                         'input2': model.input[1]},
#                               outputs = {'output':model.output})
signature = tf.saved_model.predict_signature_def(inputs={'input': keras_model.input},
                                                 outputs={'output': keras_model.output})

with tf.keras.backend.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tf.saved_model.tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()
    
print("Keras model is successfully converted using SavedGraphBuilder to", saved_graph_dir)

W0903 23:51:31.835143 140542846830400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Keras model is successfully converted using SavedGraphBuilder to ../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001


In [13]:
%%bash

CONVERT_TF_SAVED_GRAPH_DIR=../../../tmp/trt_end_to_end/convert/tf/saved_graph
find ${CONVERT_TF_SAVED_GRAPH_DIR} -type f | sort

../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/saved_model.pb
../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/variables/variables.data-00000-of-00001
../../../tmp/trt_end_to_end/convert/tf/saved_graph/resnet50/001/variables/variables.index
